In [1]:
from google.colab import drive
drive.mount('/content/drive/')
!ls

# change working directory
import os
# %cd ../
os.chdir("drive/My Drive/Covid-19")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/
drive  sample_data


In [2]:
%cd tweets
!wget -O covid-tweets-jan-mar.csv.gz https://drive.google.com/uc?id=1zLwTxMNOyyQ03ae0zFeS2xpD1hhFRpF9&export=download
%cd ..

/content/drive/My Drive/Covid-19/tweets
--2020-04-25 04:15:41--  https://drive.google.com/uc?id=1zLwTxMNOyyQ03ae0zFeS2xpD1hhFRpF9
Resolving drive.google.com (drive.google.com)... 74.125.23.102, 74.125.23.101, 74.125.23.100, ...
Connecting to drive.google.com (drive.google.com)|74.125.23.102|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-10-ao-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/1k57dr6gomvgpqfn7vr58o08uk01keor/1587788100000/09819436844883370984/*/1zLwTxMNOyyQ03ae0zFeS2xpD1hhFRpF9 [following]
--2020-04-25 04:15:43--  https://doc-10-ao-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/1k57dr6gomvgpqfn7vr58o08uk01keor/1587788100000/09819436844883370984/*/1zLwTxMNOyyQ03ae0zFeS2xpD1hhFRpF9
Resolving doc-10-ao-docs.googleusercontent.com (doc-10-ao-docs.googleusercontent.com)... 74.125.203.132, 2404:6800:4008:c03::84
Connecting to doc-10-ao-docs.googleusercontent.com (doc-10-a

In [3]:
!pip install pandarallel

# importing libraries
from bs4 import BeautifulSoup
import urllib.request
import re
import pandas as pd
from tqdm import tqdm
import datetime
from pandarallel import pandarallel
from get_tweets_by_id import getTweetTextAndLocation
pandarallel.initialize()


  Created wheel for pandarallel: filename=pandarallel-1.4.8-cp36-none-any.whl size=16112 sha256=79a7d449e6714545916ec2c839a4d6623e56fe1e3172545fa2020d888fec6ca5
  Stored in directory: /root/.cache/pip/wheels/75/a2/85/b45be2e86d86e9ec5da6d05c4b994d18c81abe76e3f39415aa
Successfully built pandarallel
INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [0]:

filepath = 'tweets/covid-tweets-jan-mar.csv.gz'

dateparse = lambda x: pd.datetime.strptime(x, '%Y-%m-%d')

str_date = '2020-03-13' # change in 270873
date = datetime.datetime.strptime(str_date, '%Y-%m-%d')

df = pd.read_csv(filepath, compression='gzip', parse_dates=['date'], date_parser=dateparse)
df = df[df['date'] == date]
df.to_csv('temp.csv', index=False)

df_reader = pd.read_csv('temp.csv', chunksize=10000, parse_dates=['date'], date_parser=dateparse)

savepath = 'covid-tweets-' + str_date + '.csv.gz'
for i, chunk in tqdm(enumerate(df_reader)):
  
    chunk['extractions'] = chunk.parallel_apply(getTweetTextAndLocation, axis=1)
    
    # Organise columns
    chunk  = chunk.merge(chunk.extractions.apply(
        lambda item: pd.Series([item[0], item[1], item[2], item[3], int(item[4]), int(item[5]), int(item[6])],
                                index=['text', 'lang', 'location_id', 'location', 'replies', 'retweets', 'likes'])), left_index=True, right_index=True)
    chunk.drop(['extractions'], axis=1, inplace=True)
    chunk = chunk[chunk['text'] != '']

    if i == 0:
      chunk.to_csv(savepath, index=False, compression='gzip', mode='w')
    else:
      chunk.to_csv(savepath, index=False, compression='gzip', mode='a', header=False)


